<a href="https://colab.research.google.com/github/legobitna/DeepLearning-basic/blob/main/7_4a_FTMLE_Introduction_to_TensorFlow_Lecture_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TensorFlow

![](https://camo.githubusercontent.com/0905c7d634421f8aa4ab3ddf19a582572df568e1/68747470733a2f2f7777772e74656e736f72666c6f772e6f72672f696d616765732f74665f6c6f676f5f736f6369616c2e706e67)


## Tensorflow High level API

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

*Load and prepare MNIST dataset:*

In [ ]:
# Load the MNIST digit dataset
mnist = tf.keras.datasets.mnist                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
# Plot some train samples
import numpy as np
import matplotlib.pyplot as plt

# Some some random images and their labels
fig, ax = plt.subplots(2, 6, figsize = (12,8))
for i in range(2):
  for j in range(6):
    index = np.random.randint(0, len(X_train)) 
    ax[i,j].imshow(X_train[index], cmap='Greys') # you can use cmap='gray' for another display color map
    ax[i,j].set_title(f'Label: {y_train[index]}')

plt.show()

In [ ]:
X_train.shape

*Build the tf.keras.Sequential model by stacking layers:*

In [ ]:
# Create a Deep Neural Network
def create_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='Adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_model()

In [ ]:
# View the structure of the network
model.summary()

*Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:*

In [ ]:
# Train the model with Train data
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model with Test data
model.evaluate(X_test, y_test)

In [ ]:
# Visualize model history
def plot_history(history, key='loss'):
    plt.figure(figsize=(12,8))

    val = plt.plot(history.epoch, history.history['val_'+key],'--', label=key.title() +' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(), label=key.title() + ' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

plot_history(history, key='accuracy')

**Cool, how can I export this model. I have to deploy it on a Flask app for my weekly/final project!**

*I will show you how to Save Checkpoints during training. You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) callback allows to continually save the model both during and at the end of training.*

*Create a tf.keras.callbacks.ModelCheckpoint callback that saves weights only during training:*

In [ ]:
checkpoint_path = "my_model.h5"

# Create a callback that saves the model's weights
# by default it saves the weights every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 save_best_only=True,
                                                 verbose=1)


In [ ]:
# Train the model with the new callback
model = create_model()
history = model.fit(X_train, 
                    y_train,  
                    epochs=10,
                    validation_data=(X_test, y_test),
                    callbacks=[cp_callback])  # Pass callback to training

*Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):*

In [ ]:
# Create a basic model instance
new_model = create_model()

# Evaluate the model
loss, acc = new_model.evaluate(X_test, y_test)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

*Then load the weights from the checkpoint and re-evaluate*

In [ ]:
# Loads the weights
new_model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = new_model.evaluate(X_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model_2 = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model_2.summary()
# .compile 이게 없으면 정확도 10프로 밖에안나옴 기존에 트레인된 모델 my_model.h5를 썻음에도 불구하고 
new_model_2.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Re-evaluate the model
loss, acc = new_model_2.evaluate(X_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
new_model_2.predict(X_test)

In [ ]:
y_test

**That's only the model's weights. How can I save the entire model**

*The model and optimizer can be saved to a file that contains both their state (weights and variables) and the model configuration. This allows you to export a model so it can be used without access to the original Python code. Since the optimizer-state is recovered, you can resume training from exactly where you left off.*

*Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))*

In [ ]:
# Create a new model instance
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=5)

# Save the entire model to a HDF5 file
model.save('my_model.h5')

*Now, recreate the model from that file:*

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

*Check its accuracy:*

In [ ]:
loss, acc = new_model.evaluate(X_test, y_test)
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Practice (Time to shine!)

In [ ]:
# Load data
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# normalize the images
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
# Plot some train samples
import numpy as np
import matplotlib.pyplot as plt

# Some some random images and their labels
fig, ax = plt.subplots(2, 6, figsize = (12,8))
for i in range(2):
  for j in range(6):
    index = np.random.randint(0, len(X_train)) 
    ax[i,j].imshow(X_train[index], cmap='Greys') # you can use cmap='gray' for another display color map
    ax[i,j].set_title(f'Label: {y_train[index]}')

plt.show()
X_train.shape

In [ ]:
# can you follow the above steps and build a model to get evaluation accuracy up to 90% for fashion MNIST for validation?
# Create a new model instance
def create_model():
    model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(10, activation='softmax')
        ])


    model.compile(optimizer='Adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test))


In [ ]:
# evaluate your model with X_test and y_test
# Re-evaluate the model
loss, acc = model.evaluate(X_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Transfer Learning and Preprocessing with Tensorflow

###Dealing with image paths

In [ ]:
import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)

We use Python’s pathlib module pretty much anytime you need to work with files in Python since we do not need to deal with different syntax for paths in different Operating Systems (like Windows, Linux and MacOS). For example, in Windows, we have "C:\Users\Tom\Downloads" but in Linux, we have "\/Users\/Tom\/Downloads" so hard-coding "/" or "\\" won't work for both of them. So pathlib is here to save our world!

The pathlib module replaces many of these filesystem-related os utilities with methods on the Path object.

In [ ]:
data_root = pathlib.Path(data_root_orig)
print(data_root)

*After downloading 218MB, you should now have a copy of the flower photos available:*

In [ ]:
for item in data_root.iterdir():
  print(item)

The pathlib.Path modules aren’t the only filepath/filesystem-related utilities in the Python standard library. The **glob** module is another handy path-related module.

We can use the glob.glob function for finding files that match a certain pattern:

```
An asterisk (*) matches zero or more characters in a segment of a name. For example, dir/*

dir/file.txt
dir/file1.jpg
dir/random.txt
dir/subdir
```

```
To list files in a subdirectory, you must include the subdirectory in the pattern, like ('dir/*/*'):
dir/subdir1/file.txt
dir/subdir1/b.jpg
dir/subdir2/random.txt
dir/subdir3/test.png
```

This is where data_root path currently is:

In [ ]:
data_root

In [ ]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
all_image_paths[:10]

In [ ]:
# number of images
image_count = len(all_image_paths)
image_count

**That's only a list of paths to the files. How can I have a quick look so I know what I'm dealing with:**

In [ ]:
import IPython.display as display

for n in range(3):
    image_path = random.choice(all_image_paths)
    print(image_path)
    display.display(display.Image(image_path))
    print()

###Where/What are the labels?

*List the available labels:*

In [ ]:
for item in data_root.glob('*/'):
  print(item, '-> Folder :',item.is_dir())

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

*Assign an index to each label:*

label_to_index will be useful to assign each image to a index label based on its folder name!

In [ ]:
# short way
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

In [ ]:
# long way
label_to_index={}
for index, name in enumerate(label_names):
  label_to_index[name] = index

label_to_index

This index_to_label will be useful for prediction!


In [ ]:
index_to_label = {v: k for k, v in label_to_index.items()}
index_to_label

*Create a list of label index:*

In [ ]:
# testing
# get the first path
one_path = all_image_paths[0]
print(one_path)

In [ ]:
# get the parent path
pathlib.Path(one_path).parent

In [ ]:
# get the parent folder name
parent_folder_name = pathlib.Path(one_path).parent.name
print(parent_folder_name)

In [ ]:
# map the parent folder to index from our label_to_index dictionary
label_to_index[parent_folder_name]

In [ ]:
# all in one go
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])
# print("First 10 paths: ")
# all_image_paths[:10]

**Cool, now how can I preprocess the images before training?**

*TensorFlow includes all the tools you need to load and process images. Here is the raw data:*


In [ ]:
img_path = all_image_paths[0]
img_raw = tf.io.read_file(img_path)
img_raw

*Decode it into an image tensor:*

In [ ]:
img_tensor = tf.image.decode_jpeg(img_raw, channels=3)

print(img_tensor.shape)
print(img_tensor.dtype)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_tensor)

*Resize it for your model:*

In [ ]:
img_final = tf.image.resize(img_tensor, [192, 192])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.dtype)

In [ ]:
plt.imshow(img_final)

*Wrap up these up in a simple functions:*

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [ ]:
import matplotlib.pyplot as plt

image_path = all_image_paths[3101]
label = all_image_labels[3101]

plt.imshow(load_and_preprocess_image(image_path))
plt.title(label_names[label])
plt.show()

With our current dataset, it got 3670 images with each size 192x192x3 = 405,872,640 bytes = 400 MB or 0.4 GB which is fitted in our either RAM or GPU Memory. No problem!

However, oftenly other image datasets are too large (10 million images with 1000x1000x3 size) to contains all the images in the memory :(( This is impossible for have the whole dataset stored in one go inside the memory!

However, we can store a list of file paths in our RAM easily (only 3670 strings)

So let's build an input pipeline for our ML model. A input pipeline takes in the raw data (our path), processes it (load, decode image, normalize it) and then feeds it to the model when the model requires for one mini-batch of data to train. 

So let's use **tf.data.Dataset** !

![](https://i.pinimg.com/originals/b8/c4/04/b8c404b584f1804512bc19e7be91e222.gif)



## Bulding pipeline with tf.data.dataset

*The easiest way is to build a `tf.data.Dataset`*

The `tf.data.Dataset` API supports writing descriptive and efficient input pipelines. Dataset usage follows a common pattern:

- Create a source dataset from your input data.
- Apply dataset transformations to preprocess the data.
- Iterate over the dataset and process the elements.
Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

More importantly, tf dataset will make the pipeline faster:

**Without tf.dataset**

![](https://miro.medium.com/max/2028/1*xJBrmPZm0LPDkQXt8WBXuQ.png)

![](https://images.viblo.asia/606e25b5-bb38-444f-bddc-a45f4f7a2f34.png)

**With tf.dataset**

![](https://miro.medium.com/max/1400/1*TiZczQ0eSR6EX50xBeXnSg.png)


![](https://images.viblo.asia/815bfa69-8363-48f4-bf7a-779d81c1a9d7.png)

**The example of tf dataset pipeline**

![](https://pic1.zhimg.com/80/v2-3b242922714b46837ac7a5ca55c63638_1440w.jpg)



### Examples with different methods of the tf.data.dataset

#### from_tensor_slices

Creates a Dataset whose elements are slices of the given tensors.

In [ ]:
# Slicing a 1D tensor produces scalar tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
list(dataset.as_numpy_iterator())

In [ ]:
# Slicing a 2D tensor produces 1D tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
list(dataset.as_numpy_iterator())

#### map

Maps map_func across the elements of this dataset.


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
def add_one(ele):
  return ele + 1
dataset = dataset.map(add_one)
list(dataset.as_numpy_iterator())

#### batch

Combines consecutive elements of this dataset into batches.

In [ ]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3)
list(dataset.as_numpy_iterator())

The components of the resulting element will have an additional outer dimension, which will be batch_size (or N % batch_size for the last element if batch_size does not divide the number of input elements N evenly and drop_remainder is False). If your program depends on the batches having the same outer dimension, you should set the drop_remainder argument to True to prevent the smaller batch from being produced

In [ ]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3, drop_remainder=True)
list(dataset.as_numpy_iterator())

#### repeat

Repeats this dataset so each original value is seen count times.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.repeat(3)
list(dataset.as_numpy_iterator())

#### shuffle

Randomly shuffles the elements of this dataset.

This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.

For instance, if your dataset contains 10,000 elements but buffer_size is set to 1,000, then shuffle will initially select a random element from only the first 1,000 elements in the buffer. Once an element is selected, its space in the buffer is replaced by the next (i.e. 1,001-st) element, maintaining the 1,000 element buffer.

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=False)
print(list(dataset.as_numpy_iterator()))
print(list(dataset.as_numpy_iterator()))

reshuffle_each_iteration controls whether the shuffle order should be different for each epoch. 

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=False)
print(list(dataset.as_numpy_iterator()))
print(list(dataset.as_numpy_iterator()))

### shuffle and repeat order, shuffle buffer size, reshuffle_each_iteration best practice

Spot the difference?

full buffer size, shuffle then repeat, reshuffle_each_iteration=True

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.shuffle(5, reshuffle_each_iteration=True)
dataset = dataset.repeat(3)
print(list(dataset.as_numpy_iterator()))

full buffer size, shuffle then repeat, reshuffle_each_iteration=False

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.shuffle(5, reshuffle_each_iteration=False)
dataset = dataset.repeat(3)
print(list(dataset.as_numpy_iterator()))

full buffer size, repeat then shuffle, reshuffle_each_iteration=True

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(5, reshuffle_each_iteration=True)
print(list(dataset.as_numpy_iterator()))

full buffer size, repeat then shuffle, reshuffle_each_iteration=False

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(5, reshuffle_each_iteration=False)
print(list(dataset.as_numpy_iterator()))

small buffer size, shuffle then repeat, reshuffle_each_iteration=True

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.shuffle(2, reshuffle_each_iteration=True)
dataset = dataset.repeat(3)
print(list(dataset.as_numpy_iterator()))

small buffer size, shuffle then repeat, reshuffle_each_iteration=False

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.shuffle(2, reshuffle_each_iteration=False)
dataset = dataset.repeat(3)
print(list(dataset.as_numpy_iterator()))

small buffer size, repeat then shuffle, reshuffle_each_iteration=True

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(2, reshuffle_each_iteration=True)
print(list(dataset.as_numpy_iterator()))

small buffer size, repeat then shuffle, reshuffle_each_iteration=False

In [ ]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(2, reshuffle_each_iteration=False)
print(list(dataset.as_numpy_iterator()))

#### skip

Creates a Dataset that skips count elements from this dataset.

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.skip(7)
list(dataset.as_numpy_iterator())

#### take

Creates a Dataset with at most count elements from this dataset.

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.take(3)
list(dataset.as_numpy_iterator())

### Use tf.data.dataset to build the pipeline in flower dataset

In [ ]:
# Testing: Slicing the array of strings, results in a dataset of strings (convert to tensor type for tf.dataset)
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [ ]:
path_ds

*Now create a new dataset that loads and formats images on the fly by mapping `load_preprocess_image` over the dataset of paths.*

In [ ]:
# This is where the pipeline starts
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

ds_map = ds.map(load_and_preprocess_from_path_label)

# add cache to increase the speed
ds_map_cache = ds.map(load_and_preprocess_from_path_label).cache()
ds_map_cache

*To train a model with this dataset, you will want the data:*
* *To be well shuffeled*
* *To be batched*
* *To repeat forever*
* *Batches to be available as soon as possible*

In [ ]:
# Let trainset is 70%, validation is 30%
train_size = int(0.7 * image_count)
val_size = int(0.3 * image_count)

# shuffle time, since the number of image paths this time is not too large so we can shuffle the whole list
# Setting a shuffle buffer size as large as the dataset ensures that the data is completely shuffled.
#ds_map_cache = ds_map_cache.shuffle(buffer_size = image_count)

# Split into train and test set
train_dataset = ds_map_cache.take(train_size)
test_dataset = ds_map_cache.skip(train_size)


# Mini-batch
BATCH_SIZE = 32

# add autotune means asking tensorflow to figure out the best settings for your VM/laptop running
AUTOTUNE = tf.data.experimental.AUTOTUNE

# `repeat` will allows the dataset to be repeat forever
# `prefetch` lets the dataset fetch batches in the background with CPU while the model is training on GPU.
SHUFFLE_BUFFER_SIZE= 1024
final_train_dataset = train_dataset.repeat().shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
final_test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

> There are a few things to note here:

> The order is important.

> * A .shuffle after a .repeat would shuffle items across epoch boundaries (some items will be seen twice before others are seen at all).

> * A .shuffle after a .batch would shuffle the order of the batches, but not shuffle the items across batches.

> * You use a buffer_size the same size as the dataset for a full shuffle. Up to the dataset size, large values provide better randomization, but use more memory.

> * The shuffle buffer is filled before any elements are pulled from it. So a large buffer_size may cause a delay when your Dataset is starting.

> * The shuffeled dataset doesn't report the end of a dataset until the shuffle-buffer is completely empty. The Dataset is restarted by .repeat, causing another wait for the shuffle-buffer to be filled.

Utimately, here is the order you should try to follow:

For train dataset -> Map -> Cache -> Repeat -> Shuffle -> Batch -> Prefetch

For test dataset -> Map -> Cache (optional) -> Batch -> Prefetch (optional)



**Awesome! Finally, now we need a model right!**


#Transfer Learning

Let's cheat by using transfer learning to give our model a big boost without putting effort to train or tune our model! 

It is a popular approach in deep learning where pre-trained models are used as the starting point on computer vision and natural language processing tasks given the vast compute and time resources required to develop neural network models on these problems and from the huge jumps in skill that they provide on related problems.

*It's fine, eveything is new on the first day. And you will see that I'm very "user-friendly".*

![](https://miro.medium.com/max/441/1*TIMA09tVqZe7tA6DckoP6g.png)

![](https://cdn-media-1.freecodecamp.org/images/nhxsEn9S-VwNdFKCClwfeKhKmTd1buwzF3pR)

------------------------------------------

![](https://www.mathworks.com/discovery/transfer-learning/_jcr_content/mainParsys/image.adapt.full.high.jpg/1576731630627.jpg)

---------------------------------------------

![](https://i.stack.imgur.com/d0iwP.png)

--------------------------------------------------

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Model_Timeline.png)

*Okay we've learned how to build a model. Let's jump a big step ahead and I'll show you a simple transfer learning example:*

*Fetch a copy of MobileNet v2 from [tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications)*

In [ ]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable=False  # freeze all the layers of the pre-trained model so their weights and bias won't be updated during gradient descent steps

*Build a model wrapped around MobileNet and use [tf.keras.layers.GlobalAveragePooling2D](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/GlobalAveragePooling2D) to average over those space dimensions before the output tf.keras.layers.Dense layer:*

In [ ]:
len(mobile_net.layers)

In [ ]:
mobile_net.summary()

In [ ]:
for i in mobile_net.layers:
  print(i)

In [ ]:
model = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(label_names), activation = 'softmax')])

In [ ]:
model.summary()

*Compile the model to describe the training procedure:*

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()
history = model.fit(final_train_dataset,validation_data=final_test_dataset, epochs=10, steps_per_epoch=steps_per_epoch)

In [ ]:
model.save('my_model.h5')

The prediction time!

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(fn)
  path = '/content/' + fn
  flower = load_and_preprocess_image(path)

  # reshape it to match with the trained images
  reshaped_flower = tf.reshape(flower, [1,192,192,3])

  # forward propagation :))
  prediction = model.predict(reshaped_flower)

  # get the index for the highest probability
  label_index = tf.math.argmax(prediction[0])

  # select the label from the index
  print("The prediction is", index_to_label[label_index.numpy()])